<a href="https://colab.research.google.com/github/acvats/Medical-Drugs--NLP/blob/master/Sentiment_Analysis_NLP_without_dataset_balancing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://www.kaggle.com/arbazkhan971/analyticvidhyadatasetsentiment/metadata

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/.kaggle"

In [2]:
!pip install kaggle

In [3]:
!mkdir .kaggle

In [4]:
cp '/content/drive/My Drive/kaggle auth key/kaggle.json' '/content/.kaggle/kaggle.json'

In [5]:
!chmod 600 /content/.kaggle/kaggle.json

In [6]:
!kaggle config set -n path -v{/content}


- path is now set to: {/content}


In [7]:
!kaggle datasets list

ref                                                    title                                              size  lastUpdated          downloadCount  
-----------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
anmolkumar/health-insurance-cross-sell-prediction      Health Insurance Cross Sell Prediction 🏠 🏥          6MB  2020-09-11 18:39:31            805  
shivan118/healthcare-analytics                         Healthcare Analytics                                2MB  2020-09-13 17:40:05            473  
Cornell-University/arxiv                               arXiv Dataset                                     885MB  2020-09-14 16:54:47           2831  
nipunarora8/age-gender-and-ethnicity-face-data-csv     AGE, GENDER AND ETHNICITY (FACE DATA) CSV          63MB  2020-09-02 13:46:38            271  
datatattle/covid-19-nlp-text-classification            Coronavirus tweets NLP - Text Classification       

In [8]:
!kaggle datasets download -d 'arbazkhan971/analyticvidhyadatasetsentiment'

 74% 5.00M/6.72M [00:00<00:00, 51.5MB/s]
100% 6.72M/6.72M [00:00<00:00, 43.0MB/s]


In [9]:
cd {/content}/datasets/

/content/{/content}/datasets


In [10]:
cd arbazkhan971/analyticvidhyadatasetsentiment

/content/{/content}/datasets/arbazkhan971/analyticvidhyadatasetsentiment


In [11]:
!mkdir data

In [12]:
!unzip analyticvidhyadatasetsentiment.zip -d data

Archive:  analyticvidhyadatasetsentiment.zip
  inflating: data/sample_submission_i5xnIZD-1.csv  
  inflating: data/test_tOlRoBf.csv   
  inflating: data/train_F3WbcTw.csv  


In [13]:
cd data

/content/{/content}/datasets/arbazkhan971/analyticvidhyadatasetsentiment/data


In [14]:
ls

sample_submission_i5xnIZD-1.csv  test_tOlRoBf.csv  train_F3WbcTw.csv


In [15]:
import pandas as pd
import numpy as np

In [16]:
train = pd.read_csv("train_F3WbcTw.csv")
test = pd.read_csv("test_tOlRoBf.csv")
sample_data = pd.read_csv("sample_submission_i5xnIZD-1.csv")

In [17]:
train.head()

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1


In [18]:
import re
def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 
 return text

In [19]:
train['text'] = train['text'].apply(cleanTxt)

In [20]:
train.head()

,unique_hash,text,drug,sentiment
0,2e180be4c9214c1f5ab51fd8cc32bc80c9f612e0,Autoimmune diseases tend to come in clusters. ...,gilenya,2
1,9eba8f80e7e20f3a2f48685530748fbfa95943e4,I can completely understand why you’d want to ...,gilenya,2
2,fe809672251f6bd0d986e00380f48d047c7e7b76,Interesting that it only targets S1P-1/5 recep...,fingolimod,2
3,bd22104dfa9ec80db4099523e03fae7a52735eb6,"Very interesting, grand merci. Now I wonder wh...",ocrevus,2
4,b227688381f9b25e5b65109dd00f7f895e838249,"Hi everybody, My latest MRI results for Brain ...",gilenya,1


In [21]:
y = train['sentiment']
y

0       2
1       2
2       2
3       2
4       1
       ..
5274    0
5275    2
5276    2
5277    2
5278    0
Name: sentiment, Length: 5279, dtype: int64

In [22]:
drug = train['drug']
drug

0          gilenya
1          gilenya
2       fingolimod
3          ocrevus
4          gilenya
           ...    
5274        alimta
5275       entyvio
5276        opdivo
5277      lucentis
5278       ocrevus
Name: drug, Length: 5279, dtype: object

In [23]:
hashkey=train['unique_hash']
train= train.drop(['unique_hash'], axis=1)

In [24]:
train = train.drop(['sentiment'], axis =1)

In [25]:
train = train.drop(['drug'], axis =1)

In [26]:
train.head()

,text
0,Autoimmune diseases tend to come in clusters. ...
1,I can completely understand why you’d want to ...
2,Interesting that it only targets S1P-1/5 recep...
3,"Very interesting, grand merci. Now I wonder wh..."
4,"Hi everybody, My latest MRI results for Brain ..."


In [27]:
import re
# Dictionary of English Contractions
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
  def replace(match):
    return contractions_dict[match.group(0)]
  return contractions_re.sub(replace, text)

# Expanding Contractions in the reviews
train['text']=train['text'].apply(lambda x:expand_contractions(x))

In [30]:
train['text']=train['text'].apply(lambda x: x.lower())

In [31]:
train.head()

,text
0,autoimmune diseases tend to come in clusters. ...
1,i can completely understand why you’d want to ...
2,interesting that it only targets s1p-1/5 recep...
3,"very interesting, grand merci. now i wonder wh..."
4,"hi everybody, my latest mri results for brain ..."


In [32]:
from sklearn.metrics import confusion_matrix 

def CN(ytest, ypred):
  model_perforamnce = confusion_matrix(ytest, ypred)
  return model_perforamnce

In [33]:
from sklearn.metrics import classification_report 

def CR(ytest, ypred):
  classification = classification_report(ytest, ypred)
  return classification

In [34]:
from sklearn.metrics import accuracy_score

def acc(ytest, ypred):
  model_accuracy = accuracy_score(ytest, ypred)
  return model_accuracy

In [35]:
from sklearn.metrics import r2_score

def r2(ytest, ypred):
  r2s= r2_score(ytest, ypred)
  return r2s

In [36]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer( stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(train.text))
train_tfv =  tfv.transform(train.text)

In [38]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(train_tfv, y, test_size=0.2) 

In [39]:
X_train.shape

(4223, 39861)

In [40]:
X_test.shape

(1056, 39861)

In [41]:
y_train.shape

(4223,)

In [42]:
y_test.shape

(1056,)

In [47]:
from sklearn.linear_model import LogisticRegression

# Fitting a simple Logistic Regression on TFIDF
clfLR = LogisticRegression(C=1.0)
clfLR.fit(X_train, y_train)
predictionsP = clfLR.predict(X_test)
predictions = clfLR.predict_proba(X_test)


print("confusion matrix :"+str(CN(y_test, predictionsP)))
print("accuracy :"+str(acc(y_test, predictionsP)))
print("classification report :"+str(CR(y_test, predictionsP)))
print("r2 score :"+str(r2(y_test, predictionsP)))
print ("logloss: %0.3f " % multiclass_logloss(y_test, predictions))

confusion matrix :[[  4   4 115]
 [  0   9 158]
 [  4   8 754]]
accuracy :0.7263257575757576
classification report :              precision    recall  f1-score   support

           0       0.50      0.03      0.06       123
           1       0.43      0.05      0.10       167
           2       0.73      0.98      0.84       766

    accuracy                           0.73      1056
   macro avg       0.55      0.36      0.33      1056
weighted avg       0.66      0.73      0.63      1056

r2 score :-0.2985542558557872
logloss: 0.691 


In [48]:
 from sklearn.naive_bayes import MultinomialNB

clfMNB = MultinomialNB()
clfMNB.fit(X_train, y_train)
predictionsP = clfMNB.predict(X_test)
predictions = clfMNB.predict_proba(X_test)


print("confusion matrix :"+str(CN(y_test, predictionsP)))
print("accuracy :"+str(acc(y_test, predictionsP)))
print("classification report :"+str(CR(y_test, predictionsP)))
print("r2 score :"+str(r2(y_test, predictionsP)))
print ("logloss: %0.3f " % multiclass_logloss(y_test, predictions))

confusion matrix :[[  0   0 123]
 [  0   0 167]
 [  0   0 766]]
accuracy :0.7253787878787878
classification report :              precision    recall  f1-score   support

           0       0.00      0.00      0.00       123
           1       0.00      0.00      0.00       167
           2       0.73      1.00      0.84       766

    accuracy                           0.73      1056
   macro avg       0.24      0.33      0.28      1056
weighted avg       0.53      0.73      0.61      1056

r2 score :-0.32468615264545475
logloss: 1.590 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
from sklearn.svm import SVC
# Fitting a simple SVM
clfSVC = SVC(C=1.0, probability=True) # since we need probabilities
clfSVC.fit(X_train, y_train)
predictionsP = clfSVC.predict(X_test)
predictions = clfSVC.predict_proba(X_test)


print("confusion matrix :"+str(CN(y_test, predictionsP)))
print("accuracy :"+str(acc(y_test, predictionsP)))
print("classification report :"+str(CR(y_test, predictionsP)))
print("r2 score :"+str(r2(y_test, predictionsP)))
print ("logloss: %0.3f " % multiclass_logloss(y_test, predictions))

confusion matrix :[[  1   0 122]
 [  0   4 163]
 [  0   4 762]]
accuracy :0.7263257575757576
classification report :              precision    recall  f1-score   support

           0       1.00      0.01      0.02       123
           1       0.50      0.02      0.05       167
           2       0.73      0.99      0.84       766

    accuracy                           0.73      1056
   macro avg       0.74      0.34      0.30      1056
weighted avg       0.72      0.73      0.62      1056

r2 score :-0.31664556901786467
logloss: 0.692 


In [50]:
from sklearn import tree

clfdt = tree.DecisionTreeClassifier()
clfdt.fit(X_train, y_train)
predictionsP = clfdt.predict(X_test)
predictions = clfdt.predict_proba(X_test)


print("confusion matrix :"+str(CN(y_test, predictionsP)))
print("accuracy :"+str(acc(y_test, predictionsP)))
print("classification report :"+str(CR(y_test, predictionsP)))
print("r2 score :"+str(r2(y_test, predictionsP)))
print ("logloss: %0.3f " % multiclass_logloss(y_test, predictions))

confusion matrix :[[ 22  19  82]
 [ 15  42 110]
 [ 66  99 601]]
accuracy :0.6297348484848485
classification report :              precision    recall  f1-score   support

           0       0.21      0.18      0.19       123
           1       0.26      0.25      0.26       167
           2       0.76      0.78      0.77       766

    accuracy                           0.63      1056
   macro avg       0.41      0.40      0.41      1056
weighted avg       0.62      0.63      0.62      1056

r2 score :-0.6784718322594152
logloss: 12.789 


In [51]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB().fit(X_train.toarray(), y_train)
predictions=clf.predict(X_test.toarray())
predictionsP = clfLR.predict(X_test)
predictions = clfLR.predict_proba(X_test)


print("confusion matrix :"+str(CN(y_test, predictionsP)))
print("accuracy :"+str(acc(y_test, predictionsP)))
print("classification report :"+str(CR(y_test, predictionsP)))
print("r2 score :"+str(r2(y_test, predictionsP)))
print ("logloss: %0.3f " % multiclass_logloss(y_test, predictions))

confusion matrix :[[  4   4 115]
 [  0   9 158]
 [  4   8 754]]
accuracy :0.7263257575757576
classification report :              precision    recall  f1-score   support

           0       0.50      0.03      0.06       123
           1       0.43      0.05      0.10       167
           2       0.73      0.98      0.84       766

    accuracy                           0.73      1056
   macro avg       0.55      0.36      0.33      1056
weighted avg       0.66      0.73      0.63      1056

r2 score :-0.2985542558557872
logloss: 0.691 
